## Эксперимент №1
В данном эксперименте производится попытка извлечения n-грамм по определенным заданным критериям, которые свойственны дискурсивным маркерам

### Скачиваем *spacy_udpipe*, импортируем нужные для работы модули

In [ ]:
!pip install spacy_udpipe

     |████████████████████████████████| 304 kB 13.9 MB/s 
     |████████████████████████████████| 6.2 MB 31.9 MB/s 
     |████████████████████████████████| 660 kB 25.8 MB/s 
     |████████████████████████████████| 10.1 MB 8.6 MB/s 
     |████████████████████████████████| 42 kB 718 kB/s 
     |████████████████████████████████| 181 kB 45.3 MB/s 
     |████████████████████████████████| 457 kB 10.2 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626629 sha256=31b796eb2cebe3f650740e4b63480c237b84595c833f06d3b9e6a85dd19819f6
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalog

In [ ]:
import re
import collections
import spacy_udpipe
spacy_udpipe.download("ru")

Downloaded pre-trained UDPipe model for 'ru' language


In [ ]:
from collections import Counter

### Извлечение

Открываем текст, чистим его, оставляя точки, запятые, дефисы:

In [ ]:
with open("/content/drive/MyDrive/news_texts/2009.txt", "r", encoding = "utf-8") as f:
    text_full = f.read()

In [ ]:
cl_text = re.sub(r'[^\w\s\.\,\-]','', text_full)
cl_text = cl_text.replace(' - ', ' ')

In [ ]:
%%time
nlp = spacy_udpipe.load("ru")

CPU times: user 5.91 s, sys: 102 ms, total: 6.01 s
Wall time: 7.2 s


In [ ]:
#получение 4-грамм
bgn_4grams_lists=[sent.lower().split()[:4] for sent in cl_text.split(".")]

#сокращаем 4-граммы в зависимости от запятых: сохраняем всю часть до запятой и одно слово после нее
bgn_grams = []
for gram in bgn_4grams_lists:
    gram_to_list = ""
    for w in gram:
        if "," in w:
            gram_to_list = " ".join(gram[:gram.index(w)+2])
        else:
            gram_to_list = " ".join(gram)
    bgn_grams.append(gram_to_list)
#в общем это должно так работать, но я пока не уверена

In [ ]:
def get_grams_with_preps(grams):
    #набираем выражения с предлогом
    prep_grams = []
    for gram in grams:
        doc = nlp(gram)
        for token in doc:
            if token.pos_ == "ADP":
                prep_grams.append(doc)
    return prep_grams

In [ ]:
def get_th_grams(grams_):
    #набираем выражения с анафорами
    th_grams = []
    for gram in grams_:
        doc = nlp(gram)
        for token in doc:
            if token.lemma_ == "это" or token.lemma_ == "то":
                if token.pos_ == "PRON":
                    th_grams.append(doc)
    return th_grams

In [ ]:
%%time
th_4grams = get_th_grams(get_grams_with_preps(bgn_grams))

CPU times: user 25min 45s, sys: 12.2 s, total: 25min 57s
Wall time: 25min 54s


Нужно попробовать убрать глаголы, т.к. кажется, что они редко встречаются в коннекторах

In [ ]:
phrase_poses = {}
for gram in th_4grams:
    doc = nlp(gram)
    phrase_poses[doc] = []
    for token in doc:
        phrase_poses[doc].append(token.pos_)

In [ ]:
grams_novbs = []
for phr, poses in phrase_poses.items():
    col = []
    if "VERB" in poses:
        for token in phr:
            if token.pos_ != "VERB": 
                col.append(str(token))
        grams_novbs.append(" ".join(col))
    else:
        grams_novbs.append(str(phr))

Убираем запятые и разные аномалии

In [ ]:
cl_grams = []
for gr in grams_novbs:
    clgr = gr.replace(",", " ")
    clgr = clgr.replace("  ", " ")
    clgr = clgr.replace("   ", " ")
    clgr = clgr.replace("  как", " как")
    clgr = clgr.replace("  чтобы", " как")
    if clgr.endswith(" "):
        clgr = clgr[:-1]
    cl_grams.append(clgr)

### Что теперь?

- составляем из начала каждой n-граммы соответствующие 2, 3-граммы 
- далее считаем c-value

In [ ]:
eksp = []
for i in cl_grams:
    i_spl = i.split()
    if len(i_spl) == 4:
        twogr = " ".join(i_spl[:2])
        thrgr = " ".join(i_spl[:3])
        eksp.append(twogr)
        eksp.append(thrgr)
    elif len(i_spl) == 3:
        twogr = " ".join(i_spl[:2])
        eksp.append(twogr)

In [ ]:
th23grams = get_th_grams(get_grams_with_preps(eksp))

In [ ]:
pref_grams = cl_grams + th23grams

Обращаем внимание на концовки:

In [ ]:
fingrams = []
for gr in pref_grams:
    doc = nlp(gr)
    if doc[-1].pos_ == "NOUN" or doc[-1].pos_ == "ADP" or doc[-1].pos_ == "ADJ" or doc[-1].pos_ == "ADV" or doc[-1].pos_ == "NUM" or doc[-1].pos_ == "X" or doc[-1].text == "он":
        pass
    else:
        fingrams.append(gr)

In [ ]:
fgrams = [str(gr) for gr in fingrams]

In [ ]:
counter = collections.Counter(fgrams)

In [ ]:
cnt = {}
for k, v in counter.items():
    if v > 1: #записываем в более короткий словарь-счетчик выражения, встретившиеся больше одного раза
        cnt[k] = v

In [ ]:
import math
import operator
from collections import OrderedDict

def sort_dict_byval(somed):
    sorted_tuple = sorted(somed.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_tuple)

d_with_cdata = {}
keys = list(cnt.keys())

for k, v in cnt.items():
    vals_f_c_t = []
    c = 0
    t = 0
    for k2 in keys:
        if str(k) in str(k2):
            c += 1
            t += cnt[k2]
    vals_f_c_t.append(v)
    vals_f_c_t.append(c-1)
    vals_f_c_t.append(t-v)
    d_with_cdata[k] = vals_f_c_t


cndt_cval = {}
for cndt, c_data in d_with_cdata.items():
    f = c_data[0]
    c = c_data[1]
    t = c_data[2]
    cndt_l = len(str(cndt).split())
    if c == 0:
        cval = math.log2(cndt_l) * f
        cndt_cval[cndt] = float("%.3f" % cval)
    else:
        cval = math.log2(cndt_l) * (f - 1/c * t)
        cndt_cval[cndt] = float("%.3f" % cval)

res_ = sort_dict_byval(cndt_cval)

with open('short_cval_1eksp.txt','w', encoding="utf-8") as out:
    for key,val in res_.items():
        out.write('{}: {}\n'.format(key,val))